In [2]:
import torch
import torch.nn.utils.prune as prune
import torch.nn as nn
import datasets
import transformers
import numpy as np
import matplotlib.pyplot as plt
from transformers import AutoTokenizer
from datasets import load_dataset
from torch.utils.data import DataLoader

from transformers import BertTokenizer, BertConfig, BertModel
import random
import pandas as pd
from IPython.display import display, HTML

import pandas as pd
import torch
from torch.utils.data import TensorDataset, DataLoader
from transformers import BertTokenizer, BertConfig, BertModel
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt
import torch.nn.utils.prune as prune
import datasets
import transformers
import numpy as np
import os
import matplotlib.pyplot as plt
from transformers import AutoTokenizer
from datasets import load_dataset
from torch.utils.data import DataLoader
from transformers import BertConfig, AdamW, AutoTokenizer, get_scheduler
import datasets
from tqdm import tqdm
import matplotlib.pyplot as plt
from transformers import BertTokenizer, BertConfig, BertModel
import random
import pandas as pd
from IPython.display import display, HTML


In [4]:
def preprocess_function(examples):
    return tokenizer(examples["sentence"], padding="max_length", truncation=True)

def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)
    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, datasets.ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
    display(HTML(df.to_html()))
    
def inspect_dataloader(dataloader, num_batches=2):
    for batch_idx, batch in enumerate(dataloader):
        if batch_idx >= num_batches:
            break
        print(f"Batch {batch_idx + 1}:")
        for key, value in batch.items():
            print(f"{key}: {value.size()}")

def inspect_dataset(dataset):
    if isinstance(dataset, dict):
        for split in dataset.keys():
            print(f"Number of examples in {split}: {len(dataset[split])}")
        for split in dataset.keys():
            print(f"\nInspecting {split} split:")
            #print(f"Column names: {dataset[split].column_names}")
            #print(f"Column types: {dataset[split].features}")
            #print("\nExample data points:")
            #for i in range(min(3, len(dataset[split]))):
               # print(dataset[split][i])
    else:
        print(f"Number of examples: {len(dataset)}")
        #print(f"Column names: {dataset.column_names}")
        #print(f"Column types: {dataset.features}")
        #print("\nExample data points:")
        #for i in range(min(3, len(dataset))):
          #  print(dataset[i])

In [5]:
# TASK: SST-2 (SENTIMENT ANALYSIS)
task = "sst2"
checkpoint = "bert-base-uncased"
batch_size = 16
dataset = datasets.load_dataset("glue", task)
metric = datasets.load_metric("glue", task, trust_remote_code=True)

/var/folders/n_/7clfry555vv6fl64fyqh7jk00000gn/T/ipykernel_38342/866392637.py:6: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = datasets.load_metric("glue", task, trust_remote_code=True)


In [7]:
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
encoded_dataset = dataset.map(preprocess_function, batched=True)

/opt/anaconda3/lib/python3.11/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [9]:
encoded_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label'])
# SST dataloaders 
SST_train_dataset = encoded_dataset['train']
SST_validation_dataset = encoded_dataset['validation']
SST_train_dataloader = DataLoader(SST_train_dataset, shuffle=True, batch_size=batch_size)
SST_validation_dataloader = DataLoader(SST_validation_dataset, batch_size=batch_size)
show_random_elements(dataset["train"])

,sentence,label,idx
0,this is n't a movie ; it 's a symptom,negative,23839
1,'ll enjoy this movie,positive,66338
2,do n't come off,negative,47399
3,its essential problem,negative,13230
4,'ve never seen the deep like you see it in these harrowing surf shots,positive,10896
5,unsettling to watch as an exploratory medical procedure or an autopsy,negative,30758
6,'s so devoid of joy and energy,negative,37352
7,an engrossing and grim portrait of hookers : what they think of themselves and their clients .,positive,48242
8,the art of getting laid in this prickly indie comedy of manners and misanthropy,positive,26543
9,"a ravishing , baroque beauty",positive,64655


In [3]:
# TASK: QQP (QUESTION PAIRS)
task = "qqp"  
checkpoint = "bert-base-uncased"
batch_size = 16

dataset = datasets.load_dataset("glue", task)
metric = datasets.load_metric("glue", task, trust_remote_code=True)

def preprocess_function(examples):
    return tokenizer(examples["question1"], examples["question2"], padding="max_length", truncation=True)

tokenizer = AutoTokenizer.from_pretrained(checkpoint)
encoded_dataset = dataset.map(preprocess_function, batched=True)
encoded_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label'])

encoded_dataset['train'] = encoded_dataset['train'].select(range(67349))
QQP_train_dataset = encoded_dataset['train']
QQP_validation_dataset = encoded_dataset['validation']
QQP_train_dataloader = DataLoader(QQP_train_dataset, shuffle=True, batch_size=batch_size)
QQP_validation_dataloader = DataLoader(QQP_validation_dataset, batch_size=batch_size)

#show_random_elements(dataset["train"])

/var/folders/n_/7clfry555vv6fl64fyqh7jk00000gn/T/ipykernel_47292/4049240689.py:7: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = datasets.load_metric("glue", task, trust_remote_code=True)
/opt/anaconda3/lib/python3.11/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Map:   0%|          | 0/390965 [00:00<?, ? examples/s]

NameError: name 'show_random_elements' is not defined

In [34]:
# TASK: CoLA 
task = "cola"  
checkpoint = "bert-base-uncased"
batch_size = 16
dataset = datasets.load_dataset("glue", task)
metric = datasets.load_metric("glue", task, trust_remote_code=True)
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
def preprocess_function(examples):
    return tokenizer(examples["sentence"], padding="max_length", truncation=True)

encoded_dataset = dataset.map(preprocess_function, batched=True)
encoded_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label'])

CoLA_train_dataset = encoded_dataset['train']
CoLA_validation_dataset = encoded_dataset['validation']
CoLA_train_dataloader = DataLoader(CoLA_train_dataset, shuffle=True, batch_size=batch_size)
CoLA_validation_dataloader = DataLoader(CoLA_validation_dataset, batch_size=batch_size)
show_random_elements(dataset["train"])

print("Inspecting CoLA Train Dataloader:")
inspect_dataloader(CoLA_train_dataloader)


Generating train split:   0%|          | 0/8551 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1043 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1063 [00:00<?, ? examples/s]

/opt/anaconda3/lib/python3.11/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Map:   0%|          | 0/8551 [00:00<?, ? examples/s]

Map:   0%|          | 0/1043 [00:00<?, ? examples/s]

Map:   0%|          | 0/1063 [00:00<?, ? examples/s]

,sentence,label,idx
0,More than two students attended every seminar.,acceptable,1053
1,We spent yesterday cooking.,acceptable,6974
2,There seem three men to be in the garden.,unacceptable,8394
3,Tagalog is speaks in the Philippines.,unacceptable,4563
4,Tracy became awarded a prize.,unacceptable,7016
5,Which god the statue?,unacceptable,7750
6,I blew up it.,unacceptable,6226
7,We were glad it was over.,acceptable,4083
8,What John wants to look at now is your notes.,acceptable,7208
9,Humans love to eat owners of pigs.,acceptable,8429


Inspecting CoLA Train Dataloader:
Batch 1:
label: torch.Size([16])
input_ids: torch.Size([16, 512])
attention_mask: torch.Size([16, 512])
Batch 2:
label: torch.Size([16])
input_ids: torch.Size([16, 512])
attention_mask: torch.Size([16, 512])


In [11]:
# TASK: RTE (TEXTUAL ENTAILMENT)
task = "rte"
dataset = datasets.load_dataset("glue", task)
metric = datasets.load_metric("glue", task, trust_remote_code=True)
def preprocess_function(examples):
    return tokenizer(examples["sentence1"], examples["sentence2"], padding="max_length", truncation=True)

encoded_dataset = dataset.map(preprocess_function, batched=True)
encoded_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label'])

RTE_train_dataset = encoded_dataset['train']
RTE_validation_dataset = encoded_dataset['validation']
RTE_train_dataloader = DataLoader(RTE_train_dataset, shuffle=True, batch_size=batch_size)
RTE_validation_dataloader = DataLoader(RTE_validation_dataset, batch_size=batch_size)

show_random_elements(dataset["train"])

,sentence1,sentence2,label,idx
0,The Statue of Liberty is so big it had to be built in 300 sections.,The Statue of Liberty was built in the year 300.,not_entailment,307
1,"C&D Technologies announced that it has closed the acquisition of Datel, Inc.",C&D Technologies acquired Datel Inc.,entailment,2368
2,"A unique feature of previous Ebola outbreaks has been the relative sparing of children. For the first time, an out break of an unusual illness-Ebola haemorrhagic fever occurred in Northern Uganda - Gulu district.",Ebola Epidemic breaks out in Zaire.,not_entailment,201
3,"QNX Software Systems Ltd., a leading provider of real-time software and services to the embedded computing market, is pleased to announce the appointment of Mr. Sachin Lawande to the position of vice president, engineering services.",Sachin Lawande Named VP of Engineering Services at QNX,entailment,559
4,"On August 21, the Polish weekly Przekrój published an advertisement for Dan-Mark exercise books, bearing the logo of 4fun.tv, a Polish music/interactive TV station. Part of the advertisement contained a definition of the word ""exercise book"". The wording looked familiar, and Wikinews consulted Wikipedia, the collaborative online encyclopedia. Upon seeing the Wikipedia definition, it appeared that the entry quoted in the promotional material was identical to the initial two paragraphs of the relevant Wikipedia entry.",Readers of online encyclopedias can edit the contents.,not_entailment,1758
5,"In a game ripe with controversy, Manchester United walked away with a crucial away win after a Ryan Giggs free kick gave them the lead. Lille protested the goal, saying the team was setting up its wall at the time Giggs took the kick, and the team nearly left the pitch. After goading by officials, the game was resumed, but Lille said it would be making a stake for a replay with UEFA. UEFA has rejected the claim and declared the goal valid, as well as deciding to ""instigate proceedings against Lille for the improper behaviour of their players immediately after the goal.""",Lille has defeated Manchester United.,not_entailment,722
6,"Priscilla Presley greeted thousands of cheering Elvis fans at a concert Friday night, marking the 25th anniversary of the King's death, telling them ""you're helping Elvis make history again.""","Priscilla Presley told fans, ""you're helping Elvis make history again.""",entailment,1663
7,"An American-owned tugboat from Italy along with its 16 member crew was hijacked around 11 am EST on Saturday in the Gulf of Aden. The Italian Ambassador, Pierandrea Magistrati, confirmed that ""there is a boat that has been hijacked, I believe by Somali pirates."" Additionally, Shona Lowe, a spokeswoman at NATO's Northwood maritime command center reported that the Italian-flagged tugboat was hijacked this morning off of Somalia's northern coast. Lowe also confirmed that Italian government officials along with the company that owned the tugboat were trying to secure the release of the crew, ten of which were Italians.",The Gulf of Aden is located along the northern coast of Somalia.,entailment,1261
8,"The maker of a promising ovarian cancer drug said, Friday, that he stopped enrolling women in a study because several patients developed gastrointestinal holes.",A study of a promising cancer drug was stopped because of a serious side effect.,entailment,1787
9,"Wal-Mart has received a lot of negative publicity recently, including allegations that it used illegal workers and made employees work without pay during lunch breaks, as well as complaints that it generally underpays employees.",Wal-Mart complains about negative publicity.,not_entailment,2478


### Inspecting Datasets

In [6]:
#print("Inspecting SST Train Dataset:")
#inspect_dataset(SST_train_dataset)
#inspect_dataset(SST_validation_dataset)
print("Inspecting QQP Train Dataset:")
inspect_dataset(QQP_train_dataset)
inspect_dataset(QQP_validation_dataset)
print("Inspecting RTE Train Dataset:")
inspect_dataset(RTE_train_dataset)
inspect_dataset(RTE_validation_dataset)

Inspecting QQP Train Dataset:
Number of examples: 67349
Number of examples: 40430
Inspecting RTE Train Dataset:


NameError: name 'RTE_train_dataset' is not defined

### Inspecting Dataloaders

In [23]:
print("Inspecting SST Train Dataloader:")
inspect_dataloader(SST_train_dataloader)
print("Inspecting QQP Train Dataloader:")
inspect_dataloader(QQP_train_dataloader)
print("Inspecting RTE Train Dataloader:")
inspect_dataloader(RTE_train_dataloader)

Inspecting SST Train Dataloader:
Batch 1:
label: torch.Size([16])
input_ids: torch.Size([16, 512])
attention_mask: torch.Size([16, 512])
Batch 2:
label: torch.Size([16])
input_ids: torch.Size([16, 512])
attention_mask: torch.Size([16, 512])
Inspecting QQP Train Dataloader:
Batch 1:
label: torch.Size([16])
input_ids: torch.Size([16, 512])
attention_mask: torch.Size([16, 512])
Batch 2:
label: torch.Size([16])
input_ids: torch.Size([16, 512])
attention_mask: torch.Size([16, 512])
Inspecting RTE Train Dataloader:
Batch 1:
label: torch.Size([16])
input_ids: torch.Size([16, 512])
attention_mask: torch.Size([16, 512])
Batch 2:
label: torch.Size([16])
input_ids: torch.Size([16, 512])
attention_mask: torch.Size([16, 512])


## Bert Model and training

In [24]:
class BertCustomHead(nn.Module):
    def __init__(self, config, num_classes, task_type='sequence_classification'):
        super(BertCustomHead, self).__init__()
        self.bert = BertModel(config)
        self.task_type = task_type

        self.heads = {
            'sequence_classification': nn.Linear(config.hidden_size, num_classes),
            'token_classification': nn.Linear(config.hidden_size, num_classes),
            'multiple_choice': nn.Linear(config.hidden_size, 1),
            'summarization': nn.Linear(config.hidden_size, config.vocab_size)
        }

        self.loss_fns = {
            'sequence_classification': nn.CrossEntropyLoss(),
            'token_classification': nn.CrossEntropyLoss(),
            'multiple_choice': nn.BCEWithLogitsLoss(),
            'summarization': nn.CrossEntropyLoss(ignore_index=tokenizer.pad_token_id)
        }

        if task_type not in self.heads:
            raise ValueError("Invalid task type. Supported types: 'sequence_classification', 'token_classification', 'multiple_choice', 'summarization'")

    def forward(self, input_ids, attention_mask=None, token_type_ids=None, decoder_input_ids=None):
        if self.task_type == 'summarization':
            outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
            sequence_output = outputs.last_hidden_state
            logits = self.heads['summarization'](sequence_output)
            return logits
        else:
            outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
            pooled_output = outputs.pooler_output
            return self.heads[self.task_type](pooled_output)


In [32]:
def train(model, train_loader, val_loader, optimizer, num_epochs=5, device='cuda'):
    model.train()

    for epoch in range(num_epochs):
        model.train()
        total_train_loss = 0
        for batch in tqdm(train_loader, desc=f"Training Epoch {epoch+1}/{num_epochs}"):
            batch = {k: v.to(device) for k, v in batch.items()}
            optimizer.zero_grad()
            outputs = model(input_ids=batch['input_ids'], attention_mask=batch['attention_mask'])
            loss = model.loss_fns['sequence_classification'](outputs, batch['label'])
            loss.backward()
            optimizer.step()
            total_train_loss += loss.item()

        avg_train_loss = total_train_loss / len(train_loader)
        print(f"Epoch {epoch+1} - Training Loss: {avg_train_loss:.4f}")

        model.eval()
        total_val_loss = 0
        correct_predictions = 0
        with torch.no_grad():
            for batch in val_loader:
                batch = {k: v.to(device) for k, v in batch.items()}
                outputs = model(input_ids=batch['input_ids'], attention_mask=batch['attention_mask'])
                loss = model.loss_fns['sequence_classification'](outputs, batch['label'])
                total_val_loss += loss.item()

                preds = torch.argmax(outputs, dim=1)
                correct_predictions += torch.sum(preds == batch['label'])

        avg_val_loss = total_val_loss / len(val_loader)
        accuracy = correct_predictions.double() / len(val_loader.dataset)
        print(f"Epoch {epoch+1} - Validation Loss: {avg_val_loss:.4f}, Accuracy: {accuracy:.4f}")

    plt.figure(figsize=(10, 5))
    plt.plot(range(1, num_epochs + 1), train_losses, label='Training Loss')
    plt.plot(range(1, num_epochs + 1), val_losses, label='Validation Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()
    plt.title('Training and Validation Loss')
    plt.show()
    
    
def preprocess_function(examples):
    return tokenizer(examples['sentence'], truncation=True, padding='max_length', max_length=128)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


In [33]:
config = BertConfig.from_pretrained(checkpoint)
num_classes = 2  # SST-2 has binary labels: positive and negative
model = BertCustomHead(config, num_classes, task_type='sequence_classification').to(device)

optimizer = AdamW(model.parameters(), lr=2e-5)
train(model, SST_train_dataloader, SST_validation_dataloader, optimizer, num_epochs=5, device=device)

/opt/anaconda3/lib/python3.11/site-packages/transformers/optimization.py:521: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
Training Epoch 1/5:   1%|▎                 | 62/4210 [09:58<11:06:55,  9.65s/it]


KeyboardInterrupt: 

### Saving the model

In [ ]:
save_directory = "bert_finetuned_sst2"
os.makedirs(save_directory, exist_ok=True)

model_to_save = model.module if hasattr(model, 'module') else model  
model_to_save.save_pretrained(save_directory)

tokenizer.save_pretrained(save_directory)

print(f"Model, tokenizer, and config saved to {save_directory}")


In [7]:
import torch
import datasets
from transformers import BertModel, BertConfig, BertTokenizer, AdamW
from transformers import BertConfig, AdamW, AutoTokenizer, get_scheduler
from torch.utils.data import DataLoader
import torch.nn as nn
import matplotlib.pyplot as plt

# Define task and checkpoint
task = "qqp"
checkpoint = "bert-base-uncased"
batch_size = 16


def preprocess_function(examples):
    return tokenizer(examples["question1"], examples["question2"], padding="max_length", truncation=True)


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

dataset = datasets.load_dataset("glue", task)
metric = datasets.load_metric("glue", task, trust_remote_code=True)
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

encoded_dataset = dataset.map(preprocess_function, batched=True)
encoded_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label'])

encoded_dataset['train'] = encoded_dataset['train'].select(range(67349))
QQP_train_dataset = encoded_dataset['train']
QQP_validation_dataset = encoded_dataset['validation']
QQP_train_dataloader = DataLoader(QQP_train_dataset, shuffle=True, batch_size=batch_size)
QQP_validation_dataloader = DataLoader(QQP_validation_dataset, batch_size=batch_size)

# Define model
class BertCustomHead(nn.Module):
    def __init__(self, config, num_classes, task_type='sequence_classification'):
        super(BertCustomHead, self).__init__()
        self.bert = BertModel(config)
        self.task_type = task_type

        self.heads = nn.ModuleDict({
            'sequence_classification': nn.Linear(config.hidden_size, num_classes),
            'token_classification': nn.Linear(config.hidden_size, num_classes),
            'multiple_choice': nn.Linear(config.hidden_size, 1),
            'summarization': nn.Linear(config.hidden_size, config.vocab_size)
        })

        self.loss_fns = {
            'sequence_classification': nn.CrossEntropyLoss(),
            'token_classification': nn.CrossEntropyLoss(),
            'multiple_choice': nn.BCEWithLogitsLoss(),
            'summarization': nn.CrossEntropyLoss(ignore_index=tokenizer.pad_token_id)
        }

        if task_type not in self.heads:
            raise ValueError("Invalid task type. Supported types: 'sequence_classification', 'token_classification', 'multiple_choice', 'summarization'")

    def forward(self, input_ids, attention_mask=None, token_type_ids=None, decoder_input_ids=None):
        if self.task_type == 'summarization':
            outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
            sequence_output = outputs.last_hidden_state
            logits = self.heads['summarization'](sequence_output)
            return logits
        else:
            outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
            pooled_output = outputs.pooler_output
            return self.heads[self.task_type](pooled_output)

# Initialize model
config = BertConfig.from_pretrained(checkpoint)
num_classes = 2 
model = BertCustomHead(config, num_classes, task_type='sequence_classification').to(device)

# Optimizer
optimizer = AdamW(model.parameters(), lr=2e-5)

# Training function
def train(model, train_dataloader, val_dataloader, optimizer, num_epochs, device):
    losses = []  # List to store losses for plotting
    model.train()
    for epoch in range(num_epochs):
        epoch_loss = 0.0
        batch_count=0
        for  batch in train_dataloader:
            print(f"Batch count: {batch_count}")
            batch_count+=1
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['label'].to(device)

            optimizer.zero_grad()
            outputs = model(input_ids=input_ids, attention_mask=attention_mask)
            loss = model.loss_fns['sequence_classification'](outputs, labels)
            loss.backward()
            optimizer.step()

            epoch_loss += loss.item()  # Accumulate loss for this batch

        losses.append(epoch_loss / len(train_dataloader))  # Calculate average loss for the epoch
        print(f'Epoch {epoch + 1} completed, Average Loss: {epoch_loss / len(train_dataloader)}')

    return losses

# Training
losses=train(model, QQP_train_dataloader, QQP_validation_dataloader, optimizer, num_epochs=3, device=device)
# Plotting losses
plt.plot(losses, label='Training Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Training Loss over Epochs')
plt.legend()
plt.savefig('training_loss.png')


/opt/anaconda3/lib/python3.11/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/opt/anaconda3/lib/python3.11/site-packages/transformers/optimization.py:521: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Batch count: 0
Batch count: 1
Batch count: 2


KeyboardInterrupt: 